In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np

In [3]:
# 1. Load Your Dataset

df = pd.read_csv("/content/drive/MyDrive/dataset/language_proficiency.csv")

# Ensure columns exist
df = df[["full_text", "Overall"]]

# Drop rows with missing values
df = df.dropna(subset=["full_text", "Overall"])

# Reset index just in case
df = df.reset_index(drop=True)

In [4]:
# 2. Tokenizer and Dataset

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

class RegressionDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.targets = targets

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.targets[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.targets)

# Split data
train_texts, val_texts, train_scores, val_scores = train_test_split(
    df["full_text"].tolist(),
    df["Overall"].tolist(),
    test_size=0.2,
    random_state=42
)

train_dataset = RegressionDataset(train_texts, train_scores, tokenizer)
val_dataset = RegressionDataset(val_texts, val_scores, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
# 3. Load model

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
model = model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# 4. Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
# 5. Training

model.train()
for epoch in range(3):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

Epoch 1:  77%|███████▋  | 1002/1297 [1:45:27<36:08,  7.35s/it]

In [ ]:
# 6. Evaluation

model.eval()
predictions, actuals = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = outputs.logits.detach().cpu().numpy().flatten()
        labels = batch["labels"].cpu().numpy().flatten()

        predictions.extend(preds)
        actuals.extend(labels)

mse = mean_squared_error(actuals, predictions)
print(f"\nValidation MSE: {mse:.4f}")

In [ ]:
# 7. Prediction Function

def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        output = model(**inputs)
        prediction = output.logits.item()
    return prediction

# Example prediction
new_text = "The explanation in the answer was clear and fluent."
predicted_score = predict(new_text)
print(f"\nPredicted score for: '{new_text}' → {predicted_score:.2f}")

In [ ]:
# 8. Save the model

model.save_pretrained("bert_regression_model")
tokenizer.save_pretrained("bert_regression_model")
print("\nModel saved to ./bert_regression_model")